In [ ]:
# @title Setup

import asyncio
from collections.abc import Generator
import copy
import json
import random

from colabtools import adhoc_import
from colabtools import proto

with adhoc_import.Google3Head():
  from google3.net.rpc.python.contrib.dynamic_stubby import stubby2

from google3.learning.deepmind.evergreen.model_access.client.python import model_client

API_KEY = ''  # @param {type: 'string'}

: 

In [ ]:
# @title Generate scenarios

In [ ]:
SCENARIO_TEMPLATE = """
Your task is to generate a variety of meeting scenarios and user personas for
use in generating meeting transcripts. These users will ultimately interact with
an AI agent in a meeting.

An example output of content is below:

scenario: This is a meeting to get leadership input on the current plan of the Meet Voice
Agent project. The project is due to launch in June 2026 but the current scope
(large) and staffing (small) puts the timeline at risk.

participants:
Anne: Engineering leader reviewing the team's plan, wants to understand the
tradeoff between pushing out the launch date vs. cutting scope.
Bob: Product leader helping explain the key use cases for the project
Carol: Presenting the project and explaining the risks and technical complexities,
proposes pushing out the launch date but wants feedback.
Dylan: Mostly listening, asks an occasional clarifying question.

participant_names:
Anne
Bob
Carol
Dylan

Please output the list of scenarios in json format; each scenario should have
the keys ""scenario"", ""participants"", and ""participant_names"". Participants
should be output as a single string, with each participant on a new line.

All scenarios should adhere to the following guidance:
{scenario_guidance}

Specific guidance on user personas:
{user_guidance}

Please output 3 scenarios and corresponding participant personas.
"""

In [ ]:
scenario_client = model_client.ModelClient(
    model_url="gemini-api:/models/gemini-2.5-pro",
    api_key=API_KEY,
)

In [ ]:
def generate_scenario_prompt(scenario_guidance, user_guidance):
  return SCENARIO_TEMPLATE.format(
      scenario_guidance=scenario_guidance,
      user_guidance=user_guidance,
  )

In [ ]:
scenario_guidance = """
Decide on a restaurant
User wants help to decide on a restaurant.
"""

user_guidance = """
There should be {num_users} users defined.
"""

In [ ]:
scenarios_list = []
for num_users in range(1, 6):
  scenario_response = scenario_client.generate(
      generate_scenario_prompt(
          scenario_guidance, user_guidance.format(num_users=num_users)
      ),
      config=model_client.make_generation_config(temperature=1.0),
  )
  scenarios_list = scenarios_list + json.loads(
      scenario_response.as_text().replace("`", "").replace("json\n", "")
  )

In [ ]:
with open("/tmp/restaurant_scenarios.json", "w") as json_file:
  # Use json.dump() to write the data to the file
  json.dump(scenarios_list, json_file, indent=4)

%download_file /tmp/restaurant_scenarios.json

In [ ]:
for i, scenario_dict in enumerate(scenarios_list):
  print('#######################################')
  scenario = scenario_dict['scenario']
  print(f'Scenario {i}: {scenario}\n')
  participants = scenario_dict['participants']
  print(f'Participants: \n{participants}\n\n')
  participant_names = scenario_dict['participant_names']
  print(f'Participant names: {participant_names}')

# Transcript generation from scenarios

In [ ]:
AUTOUSER_PROMPT_TEMPLATE = """
You are an expert dialog simulator.
Your job is to simulate the next turn in the given dialog with the given scenario as context.
You need to select a participant from the list to make the next turn and decide what they say.
Only use participants that are found in the given list.
Participants should not compliment each other on their ideas or summarize the conversation in every turn;
most responses should be short and not overly friendly or pandering. We want
participant responses to simulate what would be said aloud in a live meeting or discussion as
much as possible.

The output format should be for example:
David: I love bananas.

# Scenario
{scenario}

# Participants:
{participants}
Gemini: an AI agent called Gemini available to provide info or help as needed.
The agent should not hallucinate or makeup information, and should use Google search when needed to find info.
Gemini does not have any context on the scenario other than what is said by users.
{agent_guidance}

# Dialog:
{dialog_history}

# Next turn:
"""

In [ ]:
agent_guidance = """
The agent should be short and snappy, and only respond when there is value to be added.
"""

In [ ]:
def format_participants_list(participants_list):
  # Shuffle participants.
  plist = participants_list.split("\n")
  plist = [p.strip() for p in plist if p.strip()]
  random.shuffle(plist)
  plist = "\n".join(plist)
  return plist


def format_dialog_history(x):
  return x


def get_autouser_prompt(
    scenario: str, agent_guidance: str, participants_list, dialog_history
):
  return AUTOUSER_PROMPT_TEMPLATE.format(
      scenario=scenario,
      agent_guidance=agent_guidance,
      participants=format_participants_list(participants_list),
      dialog_history=format_dialog_history(dialog_history),
  )


def get_autouser_turn(
    autouser_client,
    scenario: str,
    agent_guidance: str,
    participants_list: str,
    dialog_history: str,
):
  autouser_prompt = get_autouser_prompt(
      scenario, agent_guidance, participants_list, dialog_history
  )
  autouser_response = autouser_client.generate(
      autouser_prompt,
      config=model_client.make_generation_config(temperature=1.0),
  )

  turn_participant, turn_text = parse_response(autouser_response)
  return (turn_participant, turn_text)


def parse_response(response):
  turn_participant, turn_text = response.as_text().split(":", maxsplit=1)
  turn_participant = turn_participant.strip()
  turn_text = turn_text.strip()
  return turn_participant, turn_text


def extend_dialog_history(dialog_history, turn_participant, turn_text):
  return dialog_history.strip() + f"\n{turn_participant}: {turn_text}\n"

In [ ]:
NUM_TURNS = 20

In [ ]:
def generate_transcript(
    scenario: str, agent_guidance: str, participant, participant_names
):
  dialog_history = ""

  autouser_client = model_client.ModelClient(
      model_url="gemini-api:/models/gemini-2.5-pro",
      api_key=API_KEY,
  )

  participants

  for turn in range(0, NUM_TURNS):
    turn_participant, turn_text = get_autouser_turn(
        autouser_client,
        scenario,
        agent_guidance,
        participants,
        dialog_history,
    )
    last_turn = f"{turn_participant}: {turn_text}"
    print(last_turn)
    if turn_participant not in participant_names:
      print(f"Error: unknown participant: {turn_participant}")
    dialog_history = extend_dialog_history(
        dialog_history, turn_participant, turn_text
    )

  return dialog_history

In [ ]:
scenarios_with_transcripts = []
for i, scenario_dict in enumerate(scenarios_list):
  print('#######################################')
  scenario = scenario_dict['scenario']
  print(f'Scenario {i}: {scenario}\n')
  participants = scenario_dict['participants']
  print(f'Participants: \n{participants}\n\n')
  participant_names = scenario_dict['participant_names'] + ['Gemini']

  transcript = generate_transcript(
      scenario, agent_guidance, participants, participant_names
  )
  scenario_dict['transcript'] = transcript
  scenarios_with_transcripts.append(scenario_dict)

In [ ]:
with open("/tmp/restaurant_scenarios_with_transcripts.json", "w") as json_file:
  json.dump(scenarios_with_transcripts, json_file, indent=4)

%download_file /tmp/restaurant_scenarios_with_transcripts.json